# Learning Gaussian Nodes with RPCircuits

In [1]:
using RPCircuits, Random, Distributions

Random.seed!(42)

TaskLocalRNG()

First, we create a gaussian with `mean = 0.3` and `variance = 1.0` using the `Distributions` package.

In [2]:
mean, var = 0.3, 1.0
gauss = Distributions.Normal(mean, var)

Normal{Float64}(μ=0.3, σ=1.0)

Then, we generate a dataset `D` with `N` samples of the previous distribution.

In [3]:
N = 100_000

samples = Distributions.rand(gauss, N)

D = reshape(samples, length(samples), 1)

100000×1 Matrix{Float64}:
  1.0883556016042917
 -0.5798585959543994
 -0.573793482209626
 -0.4332549644348927
 -0.40219519875768045
  0.22741813195413008
 -0.6129233863399266
  0.9316208311167526
  1.7386832757114135
 -0.05416984337044606
  1.096126919278033
 -0.8753711332175869
 -0.2933950393067663
  ⋮
  1.6653626910141202
  0.38197143859308064
 -0.10460459321822108
 -0.22081326086068603
  1.4174663272233832
  2.6213309220930725
 -0.9800280316976422
  0.3459752706815864
  0.8836106784973501
 -0.860814000747588
 -0.04220185450855518
  0.15538227617934502

Using `RPCircuits`, we create a `Gaussian Node` `G` with the same `mean` and `variance` as the previous gaussian distribution. Then, we apply the `NLL` function to see the Negative Log-Likelihood of `G` w.r.t. `D`.

In [4]:
G = RPCircuits.Gaussian(1, mean, var)

println("Original model NLL = ", NLL(G, D))

Original model NLL = 1.4212436987533956


Now, we create an arbitraty `Gaussian Node`that has both `mean` and `variance` different from the distribution `gauss`. Then, we apply the `EM` algorithm to learn a better distribution.

In [5]:
G_em = RPCircuits.Gaussian(1, -0.15, 2.5)

L_em = SEM(G_em; gauss=true)

println("EM initial NLL = ", NLL(G_em, D))

for i = 1:50
    update(L_em, D; learngaussians=true, verbose=false)
end

println("EM final NLL = ", NLL(G_em, D))

println("G_em = $G_em")

EM initial NLL = 1.6179711843349411
EM final NLL = 1.4212340080384096
G_em = gaussian 1 0.2970289942983956 1.0046015042225562


Similarly to the example above, we create a `Gaussian Node` with both `mean` and `variance` differente from the distribution `gauss`. However, we apply the `Gradient Descent` algorithm in the learning process.

In [10]:
G_grad = RPCircuits.Gaussian(1, -0.15, 2.5)

L_grad = GRAD(G_grad, gauss=true)

println("Grad initial NLL = ", NLL(G_grad, D))

for i = 1:2_500
    update(L_grad, D; learningrate=0.01, learngaussians=true, verbose=false)
end

println("Grad final NLL = ", NLL(G_grad, D))
println("G_grad = $G_grad")

Grad initial NLL = 1.6179711843349411
Grad final NLL = 1.4536896433734792
G_grad = gaussian 1 0.29543688549190184 1.4738744413443041
